In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.math as tm
import numpy as np
import time
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

In [2]:
def convert2_zero_one(x):
    
    t = [tf.math.sigmoid(i) for i in x]    
    return t

def cont_bern_log_norm(lam, l_lim=0.49, u_lim=0.51):
    '''
    computes the log normalizing constant of a continuous Bernoulli distribution in a numerically stable way.
    returns the log normalizing constant for lam in (0, l_lim) U (u_lim, 1) and a Taylor approximation in
    [l_lim, u_lim].
    cut_y below might appear useless, but it is important to not evaluate log_norm near 0.5 as tf.where evaluates
    both options, regardless of the value of the condition.
    '''
    
    cut_lam = tf.where(tm.logical_or(tm.less(lam, l_lim), tm.greater(lam, u_lim)), lam, l_lim * tf.ones_like(lam))
    log_norm = tm.log(tm.abs(2.0 * tm.atanh(1 - 2.0 * cut_lam))) - tm.log(tm.abs(1 - 2.0 * cut_lam))
    taylor = tm.log(2.0) + 4.0 / 3.0 * tm.pow(lam - 0.5, 2) + 104.0 / 45.0 * tm.pow(lam - 0.5, 4)
    return tf.where(tm.logical_or(tm.less(lam, l_lim), tm.greater(lam, u_lim)), log_norm, taylor)

In [3]:
class StochasticMLP(Model):
    
    def __init__(self, hidden_layer_sizes=[100], n_outputs=10, lr=1e-3):
        super(StochasticMLP, self).__init__()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.fc_layers = [Dense(layer_size) for layer_size in hidden_layer_sizes]
        self.output_layer = Dense(n_outputs)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
        
    def call(self, x):
        
        network = []
        
        for i, layer in enumerate(self.fc_layers):
            
            logits = layer(x)
            x = tfp.distributions.Bernoulli(logits=logits).sample()
            network.append(x)

        final_logits = self.output_layer(x) # initial the weight of output layer
            
        return network
    
    def target_log_prob(self, x, h, y, is_gibbs = False, is_hmc = False, is_loss = False):
        
        # get current state
        if is_hmc:
            h_current = tf.split(h, self.hidden_layer_sizes, axis = 1)
        else:    
            h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h]
        h_current = convert2_zero_one(h_current)
        h_previous = [x] + h_current[:-1]
    
        nlog_prob = 0. # negative log probability
        
        if not is_loss:
            for i, (cv, pv, layer) in enumerate(zip(h_current, h_previous, self.fc_layers)):
            
                logits = layer(pv)
                ce = tf.nn.sigmoid_cross_entropy_with_logits(labels = cv, logits = logits)
                if not is_gibbs:
                    ce += cont_bern_log_norm(tf.nn.sigmoid(logits))
            
                nlog_prob += tf.reduce_sum(ce, axis = -1)
        
        fce = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.cast(y, tf.float32), logits=self.output_layer(h_current[-1]))
        nlog_prob += tf.reduce_sum(fce, axis = -1)
            
        return -1 * nlog_prob
    
    def gibbs_new_state(self, x, h, y):
        
        '''
            generate a new state for the network node by node in Gibbs setting.
        '''
        
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        
        in_layers = self.fc_layers
        out_layers = self.fc_layers[1:] + [self.output_layer]
        
        prev_vals = [x] + h_current[:-1]
        curr_vals = h_current
        next_vals = h_current[1:] + [y]
        
        for i, (in_layer, out_layer, pv, cv, nv) in enumerate(zip(in_layers, out_layers, prev_vals, curr_vals, next_vals)):

            # node by node
            
            nodes = tf.transpose(cv)
            prob_parents = tm.sigmoid(in_layer(pv))
            
            out_layer_weights = out_layer.get_weights()[0]
            
            next_logits = out_layer(cv)
            
            new_layer = []
            
            for j, node in enumerate(nodes):
                
                # get info for current node (i, j)
                
                prob_parents_j = prob_parents[:, j]
                out_layer_weights_j = out_layer_weights[j]
                
                # calculate logits and logprob for node is 0 or 1
                next_logits_if_node_0 = next_logits[:, :] - node[:, None] * out_layer_weights_j[None, :]
                next_logits_if_node_1 = next_logits[:, :] + (1 - node[:, None]) * out_layer_weights_j[None, :]
                
                #print(next_logits_if_node_0, next_logits_if_node_1)
                
                logprob_children_if_node_0 = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.cast(nv, dtype = tf.float32), logits=next_logits_if_node_0), axis = -1)
                
                logprob_children_if_node_1 = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.cast(nv, dtype = tf.float32), logits=next_logits_if_node_1), axis = -1)
                
                # calculate prob for node (i, j)
                prob_0 = (1 - prob_parents_j) * tm.exp(logprob_children_if_node_0)
                prob_1 = prob_parents_j * tm.exp(logprob_children_if_node_1)
                prob_j = prob_1 / (prob_1 + prob_0)
            
                # sample new state with prob_j for node (i, j)
                new_node = tfp.distributions.Bernoulli(probs = prob_j).sample() # MAY BE SLOW
                
                # update nodes and logits for following calculation
                new_node_casted = tf.cast(new_node, dtype = "float32")
                next_logits = next_logits_if_node_0 * (1 - new_node_casted)[:, None] \
                            + next_logits_if_node_1 * new_node_casted[:, None] 
                
                # keep track of new node values (in prev/curr/next_vals and h_new)
                new_layer.append(new_node)
           
            new_layer = tf.transpose(new_layer)
            h_current[i] = new_layer
            prev_vals = [x] + h_current[:-1]
            curr_vals = h_current
            next_vals = h_current[1:] + [y]
        
        return h_current
    
    def generate_hmc_kernel(self, x, y, step_size = pow(1000, -1/4)):
        
        adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn = lambda v: self.target_log_prob(x, v, y, is_hmc = True),
            num_leapfrog_steps = 2,
            step_size = step_size),
            num_adaptation_steps=int(100 * 0.8))
        
        return adaptive_hmc
    
    # new proposing-state method with HamiltonianMonteCarlo
    def propose_new_state_hamiltonian(self, x, h, y, hmc_kernel, is_update_kernel = True):
    
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        h_current = tf.concat(h_current, axis = 1)

        # run the chain (with burn-in)
        num_burnin_steps = 0
        num_results = 1

        samples = tfp.mcmc.sample_chain(
            num_results = num_results,
            num_burnin_steps = num_burnin_steps,
            current_state = h_current, # may need to be reshaped
            kernel = hmc_kernel,
            trace_fn = None,
            return_final_kernel_results = True)
    
        # Generate new states of chains
        #h_state = rerange(samples[0][0])
        h_state = samples[0][0]
        h_new = tf.split(h_state, self.hidden_layer_sizes, axis = 1) 
        
        # Update the kernel if necesssary
        if is_update_kernel:
            new_step_size = samples[2].new_step_size.numpy()
            ker_new = self.generate_hmc_kernel(x, y, new_step_size)
            return(h_new, ker_new)
        else:
            return h_new
    
    def update_weights(self, x, h, y, is_gibbs = False):
        
        with tf.GradientTape() as tape:
            loss = -1 * tf.reduce_mean(self.target_log_prob(x, h, y, is_gibbs = is_gibbs))
        
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
    
    def get_predictions(self, x):

        logits = 0.0
        for layer in self.fc_layers:
            logits = layer(x)
            x = tm.sigmoid(logits)
        
        logits = self.output_layer(x)
        probs = tm.sigmoid(logits)
        #print(probs)
        labels = tf.cast(tm.greater(probs, 0.5), tf.int32)

        return labels

In [4]:
def standard_backprop(size, data_train, data_val, epochs):
    '''
    Standard Backpropogation training
    '''
    
    batch_size = 4
    
    print("Start Standard Backprop")
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape=(2,)),
            layers.Dense(size, activation = "sigmoid"),
            layers.Dense(1, activation = "sigmoid")
        ]
    )   
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    st = time.time()
    model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])
    history = model.fit(data_train, batch_size = batch_size, epochs = epochs, validation_data = data_val)
    train_time = time.time() - st
    
    return train_time, history

In [ ]:
def hmc(size, data_train, data_val, epochs):
    '''
    HMC training
    '''
    
    print("Start HMC")
    model = StochasticMLP(hidden_layer_sizes = [size], n_outputs=1, lr = 0.01)
    network = [model.call(images) for images, labels in data_train]
    kernels = [model.generate_hmc_kernel(images, labels) for images, labels in data_train]
    
    print("Start HMC Burning")
    burnin = 1000
    tlp = []
    for i in range(burnin):
        
        if(i % 100 == 0): print("Step %d" % i)
        #network, kernels = zip(*[model.propose_new_state_hamiltonian(x, net, y, ker) 
        #           for (x, y), net, ker in zip(data, network, kernels)])
        res = []
        loss = 0.0
        for (x, y), net, ker in zip(train_ds, network, kernels):
            res.append(model.propose_new_state_hamiltonian(x, net, y, ker))
            loss += -1 * tf.reduce_sum(model.target_log_prob(x, net, y))
    
        network, kernels = zip(*res)
        tlp.append(loss / 4)
        
    
    print("Start HMC Training")
    loss_ls = []
    acc_ls = []
    start_time = time.time()
    
    for epoch in range(epochs):
    
        loss = 0.0
        acc = 0.0
        for bs, (x, y) in enumerate(data):
        
            # only one mini-batch
            model.update_weights(x, network[bs], y)
            res = [model.propose_new_state_hamiltonian(x, net, y, ker, is_update_kernel = False) \
                       for (x, y), net, ker in zip(data, network, kernels)]
            network = res
            loss += -1 * tf.reduce_mean(model.target_log_prob(x, network[bs], y, is_loss = True))
    
        preds = [model.get_predictions(images) for images, labels in data]
        train_acc = accuracy_score(np.concatenate(preds), y_train)
        loss_ls.append(loss / 4)
        acc_ls.append(train_acc)
    
        print("Epoch %d/%d: - %.4fs/step - loss: %.4f - accuracy: %.4f" 
            % (epoch + 1, epochs, (time.time() - start_time) / (epoch + 1), loss_ls[epoch], train_acc))

    train_time = time.time() - start_time
    return tlp, train_time, {"acc": acc_ls, "loss": loss_ls}

In [5]:
np.random.seed(1234)
X, Y = make_moons(1000, noise = 0.3)

# Split into test and training data
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state=73)
y_train = y_train.reshape(-1)
y_val = y_val.reshape(-1)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

2021-12-01 11:34:57.138358: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model_size = [32]
for size in model_size:
    
    epochs = 500
    time_bp, history_bp = standard_backprop(size, train_ds, val_ds, epochs)
    #tlp_burnin_hmc, time_hmc, history_hmc = hmc(size, train_ds, epochs)
    #tlp_burnin_gibbs, time_gibbs, history_gibbs = gibbs(size, train_ds, epochs)

Start Standard Backprop
Epoch 1/500
 1/25 [>.............................] - ETA: 10s - loss: 0.7183 - accuracy: 0.4062

2021-12-01 11:34:57.953574: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


25/25 [==============================] - 1s 6ms/step - loss: 0.6288 - accuracy: 0.6600 - val_loss: 0.5519 - val_accuracy: 0.7900
Epoch 2/500
25/25 [==============================] - 0s 1ms/step - loss: 0.4763 - accuracy: 0.8175 - val_loss: 0.4438 - val_accuracy: 0.8000
Epoch 3/500
25/25 [==============================] - 0s 1ms/step - loss: 0.3849 - accuracy: 0.8350 - val_loss: 0.3985 - val_accuracy: 0.7950
Epoch 4/500
25/25 [==============================] - 0s 1ms/step - loss: 0.3474 - accuracy: 0.8462 - val_loss: 0.3827 - val_accuracy: 0.7950
Epoch 5/500
25/25 [==============================] - 0s 1ms/step - loss: 0.3320 - accuracy: 0.8550 - val_loss: 0.3768 - val_accuracy: 0.8050
Epoch 6/500
25/25 [==============================] - 0s 1ms/step - loss: 0.3244 - accuracy: 0.8575 - val_loss: 0.3759 - val_accuracy: 0.8000
Epoch 7/500
25/25 [==============================] - 0s 1ms/step - loss: 0.3209 - accuracy: 0.8600 - val_loss: 0.3769 - val_accuracy: 0.8050
Epoch 8/500
25/25 [======

Epoch 59/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2603 - accuracy: 0.8950 - val_loss: 0.3131 - val_accuracy: 0.8800
Epoch 60/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2577 - accuracy: 0.8963 - val_loss: 0.3100 - val_accuracy: 0.8850
Epoch 61/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2552 - accuracy: 0.8963 - val_loss: 0.3070 - val_accuracy: 0.8850
Epoch 62/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2529 - accuracy: 0.8963 - val_loss: 0.3042 - val_accuracy: 0.8850
Epoch 63/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2507 - accuracy: 0.8988 - val_loss: 0.3016 - val_accuracy: 0.8850
Epoch 64/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2486 - accuracy: 0.9000 - val_loss: 0.2990 - val_accuracy: 0.8850
Epoch 65/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2466 - accuracy: 0.9025 - val_loss: 0.2966 - val_accuracy: 0.8900
Epoch 

25/25 [==============================] - 0s 1ms/step - loss: 0.2034 - accuracy: 0.9275 - val_loss: 0.2504 - val_accuracy: 0.9050
Epoch 117/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2032 - accuracy: 0.9287 - val_loss: 0.2503 - val_accuracy: 0.9050
Epoch 118/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2029 - accuracy: 0.9300 - val_loss: 0.2501 - val_accuracy: 0.9050
Epoch 119/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2026 - accuracy: 0.9300 - val_loss: 0.2499 - val_accuracy: 0.9050
Epoch 120/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2024 - accuracy: 0.9300 - val_loss: 0.2497 - val_accuracy: 0.9050
Epoch 121/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2022 - accuracy: 0.9300 - val_loss: 0.2496 - val_accuracy: 0.9050
Epoch 122/500
25/25 [==============================] - 0s 1ms/step - loss: 0.2019 - accuracy: 0.9300 - val_loss: 0.2494 - val_accuracy: 0.9050
Epoch 123/500

...

25/25 [==============================] - 0s 1ms/step - loss: 0.1905 - accuracy: 0.9350 - val_loss: 0.2440 - val_accuracy: 0.9100
Epoch 231/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1905 - accuracy: 0.9350 - val_loss: 0.2440 - val_accuracy: 0.9100
Epoch 232/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1904 - accuracy: 0.9350 - val_loss: 0.2439 - val_accuracy: 0.9100
Epoch 233/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1903 - accuracy: 0.9350 - val_loss: 0.2439 - val_accuracy: 0.9100
Epoch 234/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1902 - accuracy: 0.9350 - val_loss: 0.2439 - val_accuracy: 0.9050
Epoch 235/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1902 - accuracy: 0.9350 - val_loss: 0.2439 - val_accuracy: 0.9050
Epoch 236/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1901 - accuracy: 0.9350 - val_loss: 0.2439 - val_accuracy: 0.9050
Epoch 237/500

25/25 [==============================] - 0s 1ms/step - loss: 0.1868 - accuracy: 0.9350 - val_loss: 0.2428 - val_accuracy: 0.9100
Epoch 288/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1867 - accuracy: 0.9362 - val_loss: 0.2427 - val_accuracy: 0.9100
Epoch 289/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1867 - accuracy: 0.9362 - val_loss: 0.2427 - val_accuracy: 0.9100
Epoch 290/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1866 - accuracy: 0.9362 - val_loss: 0.2427 - val_accuracy: 0.9100
Epoch 291/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1865 - accuracy: 0.9362 - val_loss: 0.2426 - val_accuracy: 0.9100
Epoch 292/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1865 - accuracy: 0.9362 - val_loss: 0.2426 - val_accuracy: 0.9100
Epoch 293/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1864 - accuracy: 0.9350 - val_loss: 0.2425 - val_accuracy: 0.9100
Epoch 294/500

25/25 [==============================] - 0s 1ms/step - loss: 0.1835 - accuracy: 0.9312 - val_loss: 0.2389 - val_accuracy: 0.9150
Epoch 345/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1834 - accuracy: 0.9312 - val_loss: 0.2388 - val_accuracy: 0.9150
Epoch 346/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1833 - accuracy: 0.9312 - val_loss: 0.2387 - val_accuracy: 0.9150
Epoch 347/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1833 - accuracy: 0.9312 - val_loss: 0.2386 - val_accuracy: 0.9150
Epoch 348/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1832 - accuracy: 0.9312 - val_loss: 0.2385 - val_accuracy: 0.9150
Epoch 349/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1832 - accuracy: 0.9312 - val_loss: 0.2384 - val_accuracy: 0.9150
Epoch 350/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1831 - accuracy: 0.9312 - val_loss: 0.2383 - val_accuracy: 0.9150
Epoch 351/500

25/25 [==============================] - 0s 1ms/step - loss: 0.1805 - accuracy: 0.9337 - val_loss: 0.2333 - val_accuracy: 0.9200
Epoch 402/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1804 - accuracy: 0.9337 - val_loss: 0.2333 - val_accuracy: 0.9200
Epoch 403/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1804 - accuracy: 0.9337 - val_loss: 0.2332 - val_accuracy: 0.9200
Epoch 404/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1803 - accuracy: 0.9337 - val_loss: 0.2331 - val_accuracy: 0.9200
Epoch 405/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1803 - accuracy: 0.9337 - val_loss: 0.2330 - val_accuracy: 0.9200
Epoch 406/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1803 - accuracy: 0.9337 - val_loss: 0.2329 - val_accuracy: 0.9200
Epoch 407/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1802 - accuracy: 0.9337 - val_loss: 0.2329 - val_accuracy: 0.9200
Epoch 408/500

25/25 [==============================] - 0s 1ms/step - loss: 0.1781 - accuracy: 0.9362 - val_loss: 0.2300 - val_accuracy: 0.9200
Epoch 459/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1780 - accuracy: 0.9362 - val_loss: 0.2300 - val_accuracy: 0.9200
Epoch 460/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1780 - accuracy: 0.9362 - val_loss: 0.2300 - val_accuracy: 0.9200
Epoch 461/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1780 - accuracy: 0.9362 - val_loss: 0.2299 - val_accuracy: 0.9200
Epoch 462/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1779 - accuracy: 0.9362 - val_loss: 0.2299 - val_accuracy: 0.9200
Epoch 463/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1779 - accuracy: 0.9362 - val_loss: 0.2299 - val_accuracy: 0.9200
Epoch 464/500
25/25 [==============================] - 0s 1ms/step - loss: 0.1778 - accuracy: 0.9362 - val_loss: 0.2298 - val_accuracy: 0.9200
Epoch 465/500